# Final Project Workbook
### BIOF 309 Spring 2020 

**Author:** Ramita Karra <br>
**Last edited:** 04-23-2020

In [ ]:
pwd

### Initialize and set up package files

#### Create new a directory and directory structure for the package (slightly modified code from class)

In [ ]:
%%writefile initialize.py

from pathlib import Path
import os
import shutil 

def create_package_dir(package_name='BIOF309_RDK'):
    start_dir = Path.cwd()
    print(f"Starting in {start_dir}")

    if start_dir.name == package_name:
        os.chdir(start_dir.parent)
        package_dir = start_dir
    else:
        package_dir = start_dir / package_name
    
    if package_dir.exists():
        print("Removing old directory...")
        shutil.rmtree(package_dir)

    print(f"Creating {package_dir}...")
    package_dir.mkdir()
    print(f"The current working directory is now {package_dir}")
    os.chdir(package_dir)
    
def create_package_str(package_name='EHT_RDK'):
    Path('tests').mkdir()
    python_dir = Path(package_name)
    python_dir.mkdir()
    (python_dir / '__init__.py').touch()
    Path('setup.py').touch()
    Path('LICENSE').touch()
    Path('README.md').touch()

In [ ]:
from initialize import create_package_dir, create_package_str

create_package_dir('BIOF309_RDK')
create_package_str('EHT_RDK')

#### Add metadata and installation details (slightly modified code from class)

In [ ]:
%%writefile setup.py

import setuptools

with open("README.md", "r") as fh:
    long_description = fh.read()

setuptools.setup(
    name="EHT_RDK", 
    version="0.0.1",
    author="Ramita D. Karra",
    author_email="ramita.karra@nih.gov",
    description="A package for processing Expansion Hunter Targeted (EHT) repeat data",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="https://github.com/pypa/packaging_demo",
    packages=setuptools.find_packages(),
    classifiers=[
        "Programming Language :: Python :: 3",
        "License :: OSI Approved :: MIT License",
        "Operating System :: OS Independent",
    ],
    python_requires='>=3.6',

)

#### Modify README

In [ ]:
%%writefile README.md

# EHT_RDK
## Package Description
<br>
This aim of this package is to process raw tab-delimited output returned from the ExpansionHunter-Targeted 
software tool, used for making sequence-graph-based predictions of repeat lengths for known genetic repeat loci. 
The ultimate goal is to clean, compile, and process data for many different loci into a summary table, and to 
provide visualizations pertinent to the functional relevance of this data (i.e. number of samples containing 
repeat numbers above the pathogenic threshold for each gene).  

More information on ExpansionHunter can be found [here](https://academic.oup.com/bioinformatics/article/35/22/4754/5499079). 

#### Write license (from class)

In [ ]:
%%writefile LICENSE

Copyright (c) 2018 The Python Packaging Authority

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

### Input files

#### Create a list of file paths, consisting of all files in the directory containing raw data

In [ ]:
%%writefile input_files.py

import os
import glob
import pandas as pd

def create_input_list(directory_name):
    file_list = []

    # Check to make sure that only '.txt' files are being appended to list
    for filename in os.listdir(directory_name):
        if filename.endswith('.txt'):
            file_list.append(filename)
        else:
            print('Found non .txt file in directory: ' + filename)
            
    return(file_list)

In [ ]:
from input_files import create_input_list

# Pass directory containing raw ExpansionHunter - Targeted data
files_to_import = create_input_list('/Users/dewanr2/Documents/Ramitas_Docs/NIH_Classes/BIOF309/ExpansionHunterTargeted')

In [ ]:
display(files_to_import)

#### EXPLORATORY DATA ANALYSIS

In [ ]:
# Change into directory containing raw data

directory_name = '/Users/dewanr2/Documents/Ramitas_Docs/NIH_Classes/BIOF309/ExpansionHunterTargeted'
os.chdir(directory_name)

In [ ]:
pwd

In [ ]:
# Import first file in file_list 
test_df = pd.read_csv(file_list[0], sep='\t')

# Examine columns
print(test_df.columns)

In [ ]:
# Create 'min' and 'max' columns for minimum and maximum allele repeat values respectively
test_df['min'] = test_df[['REPCN:ATXN7_GCC_allele1','REPCN:ATXN7_GCC_allele2']].min(axis=1)
test_df['max'] = test_df[['REPCN:ATXN7_GCC_allele1','REPCN:ATXN7_GCC_allele2']].max(axis=1)

# Select only desired columns
test_df = test_df[['SampleID','chr','pos','min','max']]

print(test_df.columns)
print(test_df.info())

*EDA suggests that when importing dataframes, need to apply the following:*
- evaluate data to create 'min' and 'max' columns
- rename 'max' column with gene name
- select only desired columns: 'SampleID','chr','pos','max'

*EDA did not show any null entries for this df, but need to import with default null value in case all genes were not evaluated for all samples*

#### Import all files from list of filepaths created earlier

In [ ]:
# Confirm that current directory contains raw data

pwd

In [ ]:
# Create list of dataframes with columns: 'SampleID','chr','pos','min','max'

df_list = []

for filename in file_list:
    
    # Get gene name
    if filename.startswith("ExpansionHunterTargeted.ftd"):
        gene = filename.replace("ExpansionHunterTargeted.ftd.","", 1)
        if gene.endswith(".txt"):
            gene = gene.replace(".txt","",1)
        else:
            print("filename does not contain suffix")
    else:
        print("filename does not contain prefix")
    
    # Import and format df
    df_temp = pd.read_csv(filename, sep='\t')
    df_temp.rename(columns={ df_temp.columns[6]: "allele1" }, inplace = True)
    df_temp.rename(columns={ df_temp.columns[7]: "allele2" }, inplace = True)
    df_temp['min'] = df_temp[['allele1','allele2']].min(axis=1)
    df_temp[gene] = df_temp[['allele1','allele2']].max(axis=1)
    df_temp = df_temp[['SampleID','chr','pos',gene]]
    df_list.append(df_temp)

In [ ]:
display(df_list[0].head())

### Create dictionary of gene info

In [ ]:
# Create dictionary with keys as genes, subdictionaries as key:value pairs for "chr", "pos"

gene_dict = {}

for df in df_list:
    # Get gene name
    gene = df.columns[3]
    
    # Get chromosome
    chr_num = df.iloc[0,1]
    chr_num = chr_num.replace("chr","",1) # Remove "chr" prefix
    
    # Get gene position
    pos = df.iloc[0,2]
    
    # Edit dictionary
    gene_dict.update( {gene : {"chr":chr_num, "pos":pos}} )a

In [ ]:
display(gene_dict['C9ORF72'])

### Merge gene-specific dataframes into master dataframe